# Importer les bibliothèques nécessaires
Importer PyTorch et d'autres bibliothèques nécessaires comme torch, torch.nn, torch.optim, et matplotlib pour la visualisation.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Vérification de la disponibilité du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Définir le modèle MLP
Créer une classe pour le modèle MLP en héritant de torch.nn.Module et définir les couches dans la méthode __init__ ainsi que la méthode forward.

In [2]:
# Définir le modèle MLP
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPModel, self).__init__()
        # Définir les couches du modèle
        self.fc1 = nn.Linear(input_size, hidden_size)  # Première couche entièrement connectée
        self.relu = nn.ReLU()  # Fonction d'activation ReLU
        self.fc2 = nn.Linear(hidden_size, output_size)  # Deuxième couche entièrement connectée

    def forward(self, x):
        # Définir le passage avant
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Exemple d'initialisation du modèle (les dimensions doivent être adaptées aux données)
# input_size = taille des caractéristiques d'entrée
# hidden_size = nombre de neurones dans la couche cachée
# output_size = taille de la sortie (par exemple, nombre de classes pour une classification)
input_size = 784  # Exemple pour des images 28x28 aplaties
hidden_size = 128
output_size = 10  # Exemple pour une classification à 10 classes
model = MLPModel(input_size, hidden_size, output_size).to(device)

# Préparer les données
Charger un dataset (par exemple MNIST ou un dataset personnalisé), le normaliser et le diviser en ensembles d'entraînement et de test.

In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Définir les transformations pour normaliser les données
transform = transforms.Compose([
    transforms.ToTensor(),  # Convertir les images en tenseurs PyTorch
    transforms.Normalize((0.5,), (0.5,))  # Normaliser les images (moyenne=0.5, écart-type=0.5)
])

# Charger le dataset MNIST
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Diviser le dataset en ensembles d'entraînement et de validation
train_size = int(0.8 * len(dataset))  # 80% pour l'entraînement
val_size = len(dataset) - train_size  # 20% pour la validation
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Charger le dataset de test
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Créer des DataLoaders pour itérer sur les données
batch_size = 64  # Taille des lots
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100.0%
100.0%
100.0%
100.0%


# Configurer l'optimiseur et la fonction de perte
Définir une fonction de perte (par exemple torch.nn.CrossEntropyLoss) et un optimiseur (par exemple torch.optim.Adam).

In [5]:
# Configurer l'optimiseur et la fonction de perte

# Définir la fonction de perte
criterion = nn.CrossEntropyLoss()

# Définir l'optimiseur
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Taux d'apprentissage fixé à 0.001

# Entraîner le modèle
Écrire une boucle d'entraînement pour plusieurs époques, calculer la perte, effectuer la rétropropagation et mettre à jour les poids.

In [6]:
# Entraîner le modèle
num_epochs = 10  # Nombre d'époques

# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()  # Mettre le modèle en mode entraînement
    running_loss = 0.0  # Initialiser la perte cumulée

    for inputs, labels in train_loader:
        # Déplacer les données sur le GPU si disponible
        inputs, labels = inputs.view(inputs.size(0), -1).to(device), labels.to(device)

        # Réinitialiser les gradients
        optimizer.zero_grad()

        # Passage avant
        outputs = model(inputs)

        # Calcul de la perte
        loss = criterion(outputs, labels)

        # Rétropropagation
        loss.backward()

        # Mise à jour des poids
        optimizer.step()

        # Accumuler la perte
        running_loss += loss.item()

    # Afficher la perte moyenne pour l'époque
    print(f"Époque {epoch+1}/{num_epochs}, Perte: {running_loss/len(train_loader):.4f}")

Époque 1/10, Perte: 0.4174
Époque 2/10, Perte: 0.2306
Époque 3/10, Perte: 0.1650
Époque 4/10, Perte: 0.1325
Époque 5/10, Perte: 0.1113
Époque 6/10, Perte: 0.0954
Époque 7/10, Perte: 0.0839
Époque 8/10, Perte: 0.0768
Époque 9/10, Perte: 0.0682
Époque 10/10, Perte: 0.0631


# Évaluer le modèle
Évaluer les performances du modèle sur l'ensemble de test et afficher des métriques comme la précision ou la perte.

In [7]:
# Évaluer le modèle
model.eval()  # Mettre le modèle en mode évaluation
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():  # Désactiver la calcul des gradients pour l'évaluation
    for inputs, labels in test_loader:
        # Déplacer les données sur le GPU si disponible
        inputs, labels = inputs.view(inputs.size(0), -1).to(device), labels.to(device)

        # Passage avant
        outputs = model(inputs)

        # Calcul de la perte
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Prédictions
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculer la perte moyenne et la précision
average_loss = test_loss / len(test_loader)
accuracy = 100 * correct / total

# Afficher les résultats
print(f"Perte moyenne sur l'ensemble de test: {average_loss:.4f}")
print(f"Précision sur l'ensemble de test: {accuracy:.2f}%")

Perte moyenne sur l'ensemble de test: 0.1017
Précision sur l'ensemble de test: 97.10%
